# scipy.sparse模块-稀疏矩阵操作

考虑到需要使用一些稀疏矩阵在计算中提供帮助，需要学习一下稀疏矩阵的操作

+ csc_matrix: Compressed Sparse Column format
+ csr_matrix: Compressed Sparse Row format
+ bsr_matrix: Block Sparse Row format
+ lil_matrix: List of Lists format
+ dok_matrix: Dictionary of Keys format
+ coo_matrix: COOrdinate format (aka IJV, triplet format)
+ dia_matrix: DIAgonal format

lil_matrix和numpy类似，可以支持基础的切片与花式索引；
但是对于numpy的一些函数操作，考虑是否有scipy相应的接口，如果没有，建议使用toarray()将稀疏矩阵转换为numpy的array之后再进行操作。

+ 矩阵操作：乘法或者求逆矩阵
建议将矩阵转化为 CSC 或者 CSR；lil矩阵是基于行的，因此转换为CSR会比较好。

CSC，CSR， COO之间的转换时线性耗时的。

### Example 1

In [1]:
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
from numpy.linalg import solve, norm
from numpy.random import rand
import numpy as np

In [2]:
A = lil_matrix((1000,1000))
A[0,:100] = rand(100)
A[1,100:200] = A[0,:100]
A

<1000x1000 sparse matrix of type '<type 'numpy.float64'>'
	with 200 stored elements in LInked List format>

In [3]:
type(A)

scipy.sparse.lil.lil_matrix

In [4]:
A.setdiag(rand(1000))

In [5]:
# 将其转换为CSR格式， 求解线性方程组 A x=b
A = A.tocsr()
b = rand(1000)
%time x = spsolve(A, b)

CPU times: user 575 µs, sys: 1.17 ms, total: 1.74 ms
Wall time: 4.94 ms


In [8]:
# 将其转化为dense矩阵的耗时是多少呢?
%time x_ = solve(A.toarray(), b)

CPU times: user 54.1 ms, sys: 13.9 ms, total: 68 ms
Wall time: 50 ms


**可以看出，转化为稀疏矩阵之后，运算的耗时降低了**

In [9]:
err = norm(x-x_)
err < 1e-10

True

### Example 2
COO format

In [11]:
>>> from scipy import sparse
>>> from numpy import array
>>> I = array([0,3,1,0])
>>> J = array([0,3,1,2])
>>> V = array([4,5,7,9])
>>> A = sparse.coo_matrix((V,(I,J)),shape=(4,4))


In [14]:
>>> I = array([0,0,1,3,1,0,0])
>>> J = array([0,2,1,3,1,0,0])
>>> V = array([1,1,1,1,1,1,1])
>>> B = sparse.coo_matrix((V,(I,J)),shape=(4,4)).tocsr()

In [15]:
B

<4x4 sparse matrix of type '<type 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

### BSR format
BSR格式和CSR格式类似，不过BSR适合那些子块是dense矩阵的情况

In [17]:
from scipy.sparse import bsr_matrix
indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6]).repeat(4).reshape(6, 2, 2)
bsr_matrix((data,indices,indptr), shape=(6, 6)).toarray()

array([[1, 1, 0, 0, 2, 2],
       [1, 1, 0, 0, 2, 2],
       [0, 0, 0, 0, 3, 3],
       [0, 0, 0, 0, 3, 3],
       [4, 4, 5, 5, 6, 6],
       [4, 4, 5, 5, 6, 6]])

In [23]:
import numpy as np
from scipy.sparse import csr_matrix
csr_matrix((3, 4), dtype=np.int8).toarray()

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int8)

In [27]:
# 因此如果考虑使用稀疏矩阵存储一个数据,
# 矩阵与向量进行操作的时候，需要使用scipy.sparse.linalg.aslinearoperator吗

# 如果单纯使用稀疏矩阵来做

A = np.zeros((5000,5000))
A[0,:200] = rand(200)

In [29]:
A[100, 100:300] = rand(200)

array([[ 0.37770278,  0.51910229,  0.86195372, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [34]:
B = csr_matrix(A)

In [35]:
v = rand(5000)

%timeit A.dot(v)

100 loops, best of 3: 8.94 ms per loop


In [37]:
%time B.dot(v)

CPU times: user 66 µs, sys: 5 µs, total: 71 µs
Wall time: 76.1 µs


array([ 52.86398576,   0.        ,   0.        , ...,   0.        ,
         0.        ,   0.        ])

In [39]:
type(B.dot(v))

numpy.ndarray

In [41]:
err = norm(B.dot(v)-A.dot(v))
err

3.1776437161565096e-14

# Sparse matrix

## Diagonal Format (DIA)


In [71]:
data = np.array([[1,2,3,4]]).repeat(3,axis=0)
data[0] = [2,1,2,0]
data

array([[2, 1, 2, 0],
       [1, 2, 3, 4],
       [1, 2, 3, 4]])

In [75]:
offsets = np.array([-1,1,2])
from scipy.sparse import dia_matrix
mtx = dia_matrix((data,offsets),shape=(4,4))
mtx

<4x4 sparse matrix of type '<type 'numpy.int64'>'
	with 8 stored elements (3 diagonals) in DIAgonal format>

In [76]:
mtx.todense()

matrix([[0, 2, 3, 0],
        [2, 0, 3, 4],
        [0, 1, 0, 4],
        [0, 0, 2, 0]])

In [47]:
%timeit mtx.dot(rand(4))

The slowest run took 6.68 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.86 µs per loop


In [50]:
%timeit mtx.toarray().dot(rand(4))

10000 loops, best of 3: 98.8 µs per loop


In [51]:
dia_matrix?

In [52]:
res = mtx.todense()
res

matrix([[1, 0, 3, 0],
        [1, 2, 0, 4],
        [0, 2, 3, 0],
        [0, 0, 3, 4]])

In [54]:
dia_matrix(res).todense()

matrix([[1, 0, 3, 0],
        [1, 2, 0, 4],
        [0, 2, 3, 0],
        [0, 0, 3, 4]])

# 我的程序如何来写?

+ 先定义一个函数findbandwidth来计算窗宽
+ 再定义一个函数bandstorage来存储矩阵

In [59]:
def findbandwidth(A):
    """
返回一个矩阵的窗宽

Parameters:

A: 一个对称矩阵

Return :

窗宽 bindwidth
"""

    i, j=np.nonzero(A)
    bindwidth=max(abs(i - j))
    
    return bindwidth


In [93]:

def bandstorage(A, p):
    """
将一个对称band 矩阵进行banded 存储

Parameters:

A: 对称band矩阵
p：窗宽

Return：

B: A的压缩存储

    """

    dim=A.shape
    n=dim[1]
    B=np.zeros((p + 1, n))
    for i in range(n):  # column
        if i <= (n - p - 1):
            for j in range(i, p + i + 1):
                B[j - i + 1, i]=A[j, i]
        else:
            for j in range(n):
                B[j - i + 1, i]=A[j, i]

    return B


In [56]:
# 找一个稀疏矩阵作为例子？
# 试一试:
res = np.array([[1,2,0,0],[2,2,1,0],[0,1,3,2],[0,0,2,4]])
res

array([[1, 2, 0, 0],
       [2, 2, 1, 0],
       [0, 1, 3, 2],
       [0, 0, 2, 4]])

In [58]:
findbandwidth(res)

1

In [70]:
bandstorage(res,2)

array([[ 0.,  0.,  0.,  2.],
       [ 1.,  2.,  3.,  4.],
       [ 2.,  1.,  2.,  0.]])

In [78]:
B = np.array([[1,2,3,4],[2,1,2,10],[3,4,5,6]])
B

array([[ 1,  2,  3,  4],
       [ 2,  1,  2, 10],
       [ 3,  4,  5,  6]])

In [83]:
test=dia_matrix((B, np.array([0,1,2])), shape=(5,5))
out=test.todense()
out

matrix([[ 1,  1,  5,  0,  0],
        [ 0,  2,  2,  6,  0],
        [ 0,  0,  3, 10,  0],
        [ 0,  0,  0,  4,  0],
        [ 0,  0,  0,  0,  0]])

In [84]:
findbandwidth(out)

2

In [94]:
bandstorage(res,1)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [88]:
res


array([[1, 2, 0, 0],
       [2, 2, 1, 0],
       [0, 1, 3, 2],
       [0, 0, 2, 4]])